In [ ]:
from google.colab import files

# Paso 1: Subir los archivos desde la computadora
uploaded = files.upload()

Saving CatsDogs.zip to CatsDogs.zip


In [ ]:
from google.colab import files
import zipfile
import os



# Paso 2: Descomprimir el archivo .zip
for filename in uploaded.keys():
    if filename.endswith(".zip"):
        # Descomprimir el archivo
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("/content")  # Extrae en el directorio actual
        print(f"Archivo {filename} descomprimido exitosamente")




Archivo CatsDogs.zip descomprimido exitosamente


# **Modelo Convolucional**

In [ ]:
import os, sys, shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers, models, optimizers
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img,img_to_array,array_to_img
import random

# Fijar las semillas para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Ruta a la carpeta descomprimida
base_dir = "/content/CatsDogs/mybase"

# Directorios de entrenamiento, validación y prueba
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

print('total training '+ 'cats' +' images:', len(os.listdir(train_cats_dir)))
print('total training '+ 'dogs' +' images:', len(os.listdir(train_dogs_dir)))
print('total validation '+ 'cats' +' images:', len(os.listdir(validation_cats_dir)))
print('total validation '+ 'dogs' +' images:', len(os.listdir(validation_dogs_dir)))
print('total test '+ 'cats' +' images:', len(os.listdir(test_cats_dir)))
print('total test '+ 'dogs' +' images:', len(os.listdir(test_dogs_dir)))


# Modelo de red neuronal convolucional
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

# Compilar el modelo
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

# Generadores de datos
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flujo de datos para entrenamiento, validación y prueba
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True,
    seed=SEED
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True,
    seed=SEED
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# Entrenar el modelo sin `steps_per_epoch` para que lo maneje automáticamente
history = model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=20,
    validation_data=validation_generator)




# Graficar la precisión y la pérdida durante el entrenamiento
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Accuracy en entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Accuracy en validación')
plt.title('Accuracy en entrenamiento y validación')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Pérdida en entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Pérdida en validación')
plt.title('Pérdida en entrenamiento y validación')
plt.legend()
plt.show()

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_generator)
print(f"Accuracy en el conjunto de prueba: {test_acc}")
print(f"Pérdida en el conjunto de prueba: {test_loss}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 311ms/step - acc: 0.6443 - loss: 0.6463
Accuracy en el conjunto de prueba: 0.6370000243186951
Pérdida en el conjunto de prueba: 0.6441941857337952
